In [1]:
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [2]:
a = pd.read_excel('Swiggy_Pincode_OOS.xlsx')

In [3]:
a

,STORE_ID,Master SKU,Category,Total Qty,Total GCS,DRR_Qty,DRR_GCS,Item ID,Pincode,City,Link
0,1230878,Muesli_DC_700,Muesli,29,12954.04,0.966667,863.602667,MPUDK5M2G4,560047,Bangalore,https://www.swiggy.com/instamart/item/MPUDK5M2...
1,1231052,Muesli_DC_700,Muesli,21,9381.20,0.700000,625.413333,MPUDK5M2G4,560102,Bangalore,https://www.swiggy.com/instamart/item/MPUDK5M2...
2,1190778,PrB_BB,Protein Bar,74,8787.50,2.466667,585.833333,NFBMM07BDP,560038,Bangalore,https://www.swiggy.com/instamart/item/NFBMM07B...
3,929914,Muesli_DC_700,Muesli,18,8053.86,0.600000,536.924000,MPUDK5M2G4,560017,Bangalore,https://www.swiggy.com/instamart/item/MPUDK5M2...
4,1230878,PrB_AF,Protein Bar,65,7718.75,2.166667,514.583333,BUA1526MDK,560047,Bangalore,https://www.swiggy.com/instamart/item/BUA1526M...
...,...,...,...,...,...,...,...,...,...,...,...
3290,649837,BB_BP_Tropicana_Orange_1l,Breakfast Bar,1,40.00,0.033333,2.666667,HW0QO8NRA6,400029,Mumbai,https://www.swiggy.com/instamart/item/HW0QO8NR...
3291,1230885,BB_BP_Tropicana_Orange_1l,Breakfast Bar,1,40.00,0.033333,2.666667,HW0QO8NRA6,400007,Mumbai,https://www.swiggy.com/instamart/item/HW0QO8NR...
3292,1229996,EB_CC,Energy Bar,1,38.00,0.033333,2.533333,LKI7W4WLQZ,110056,Delhi,https://www.swiggy.com/instamart/item/LKI7W4WL...
3293,1381355,EB_CC,Energy Bar,1,38.00,0.033333,2.533333,LKI7W4WLQZ,500027,Hyderabad,https://www.swiggy.com/instamart/item/LKI7W4WL...


In [4]:
a = a.sort_values(by=["DRR_GCS"],ascending = False)

In [5]:
b = a[0:500]

In [6]:
b = b.sort_values(by=["Pincode"])

In [7]:
b.reset_index(inplace=True)

In [8]:

b.drop('index',axis=1,inplace=True)

In [9]:
b

,STORE_ID,Master SKU,Category,Total Qty,Total GCS,DRR_Qty,DRR_GCS,Item ID,Pincode,City,Link
0,969385,PrB_BB,Protein Bar,20,2375.00,0.666667,158.333333,NFBMM07BDP,110001,Noida 1,https://www.swiggy.com/instamart/item/NFBMM07B...
1,969385,Muesli_P_CAC_350,Muesli,8,2370.00,0.266667,158.000000,IBD5BTKDRP,110001,Noida 1,https://www.swiggy.com/instamart/item/IBD5BTKD...
2,969385,PrB_AF,Protein Bar,21,2493.75,0.700000,166.250000,BUA1526MDK,110001,Noida 1,https://www.swiggy.com/instamart/item/BUA1526M...
3,1267916,PrB_BB,Protein Bar,14,1662.50,0.466667,110.833333,NFBMM07BDP,110003,Delhi,https://www.swiggy.com/instamart/item/NFBMM07B...
4,1261842,Muesli_P_CAC_350,Muesli,6,1777.50,0.200000,118.500000,IBD5BTKDRP,110015,Delhi,https://www.swiggy.com/instamart/item/IBD5BTKD...
...,...,...,...,...,...,...,...,...,...,...,...
495,1167234,PB_DC400_Britannia_Bread,Peanut Butter,10,1867.50,0.333333,124.500000,C4HWUPTV99,700086,Kolkata,https://www.swiggy.com/instamart/item/C4HWUPTV...
496,1167234,Oats_DC_400,Oats,8,1584.00,0.266667,105.600000,HWKWO9PZ3A,700086,Kolkata,https://www.swiggy.com/instamart/item/HWKWO9PZ...
497,1167234,Muesli_DC_400,Muesli,9,2769.00,0.300000,184.600000,536LA2SFJW,700086,Kolkata,https://www.swiggy.com/instamart/item/536LA2SF...
498,867467,EB_CC,Energy Bar,46,1748.00,1.533333,116.533333,LKI7W4WLQZ,700089,Kolkata,https://www.swiggy.com/instamart/item/LKI7W4WL...


In [10]:
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
mobile_emulation = {
   "deviceMetrics": { "width": 360, "height": 640, "pixelRatio": 3.0 },
   "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19" }

In [11]:
options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
options.add_argument("--disable-javascript")
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')

In [12]:
count = 0
df = {"Date":[],"Master SKU":[],"Category":[],"City":[],"Pincode":[],"Selling Price":[],"Availability":[],"STORE_ID":[],"Opportunity Lost":[]}

In [13]:
start_time = time.time()

In [46]:
i = 419

In [47]:
while i < 500:
    browser = webdriver.Chrome(executable_path=path, chrome_options = options)
    browser.get(b.loc[i,"Link"])
    element = browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/button')
    element.click()
    sleep(2)
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[1]/div/div[1]/label/input').send_keys(str(b.loc[i,"Pincode"]))
    sleep(2)
    browser.find_element_by_xpath('//*[@id="root"]/div/div[1]/div[2]/div/div/button/span').click()
    sleep(5)
    x = browser.page_source
    soup = BeautifulSoup(x,"html.parser")
    soup.beautify
    
    try:
        oos = soup.find("span",{"class":"_2z1M0 _1V2V5"})
        availability = oos.text
        sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
    except AttributeError:
        try:
            oos = soup.find("div",{"class":"_1IsDA"}).text
            if oos == 'go to swiggy home':
                availability = "Out of Stock"
            sp = 0
        except AttributeError:
            try:
                sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                availability = "In Stock"
            except AttributeError:
                try:
                    ta = soup.find("div",{"class":"_2xPHa _2loQy"})
                    if ta.text == "Try Again":
                        sp = (0)
                        availability = "Out of Stock"
                except AttributeError:
                    browser.get(b.loc[i,"Link"])
                    sleep(3)
                    x = browser.page_source
                    soup = BeautifulSoup(x,"html.parser")
                    soup.beautify
                    sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                    availability = "In Stock"
                
    count = count+1
    print(count)            
    df["Selling Price"].append(sp)
    df["Availability"].append(availability)
    df["Date"].append(datetime.now())
    df['Master SKU'].append(b.loc[i,"Master SKU"])
    df["Category"].append(b.loc[i,"Category"])
    df["City"].append(b.loc[i,"City"])
    df["Pincode"].append(b.loc[i,"Pincode"])
    df["STORE_ID"].append(b.loc[i,"STORE_ID"])
    df["Opportunity Lost"].append(b.loc[i,"DRR_GCS"])
    
    j = i+1
    while b.loc[j,"Pincode"] == b.loc[j-1,"Pincode"]:
        browser.get(b.loc[j,"Link"])
        sleep(3)
        x = browser.page_source
        soup = BeautifulSoup(x,"html.parser")
        soup.beautify
        try:
            oos = soup.find("span",{"class":"_2z1M0 _1V2V5"})
            availability = oos.text
            sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text 
        except AttributeError:
            try:
                sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                availability = "In Stock"
            except AttributeError:
                try:
                    sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                    availability = "In Stock"
                except AttributeError:
                    try:
                        ta = soup.find("div",{"class":"_2xPHa _2loQy"})
                        if ta.text == "Try Again":
                            sp = (0)
                            availability = "Out of Stock"
                    except AttributeError:
                        browser.get(b.loc[j,"Link"])
                        sleep(3)
                        x = browser.page_source
                        soup = BeautifulSoup(x,"html.parser")
                        soup.beautify
                        sp = soup.find("span",{"class":"_3fCHW iD_f-"}).text
                        availability = "In Stock"
        
        df["Selling Price"].append(sp)
        df["Availability"].append(availability)
        df["Date"].append(datetime.now())
        df['Master SKU'].append(b.loc[j,"Master SKU"])
        df["Category"].append(b.loc[j,"Category"])
        df["City"].append(b.loc[j,"City"])
        df["Pincode"].append(b.loc[j,"Pincode"])
        df["STORE_ID"].append(b.loc[j,"STORE_ID"])
        df["Opportunity Lost"].append(b.loc[j,"DRR_GCS"])
        j = j + 1
        count+=1
        print(count)
    
    i = j
    browser.close()
        
    

420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500


KeyError: 500

In [48]:
fin = pd.DataFrame(df)

In [49]:
fin

,Date,Master SKU,Category,City,Pincode,Selling Price,Availability,STORE_ID,Opportunity Lost
0,2023-03-30 14:59:24.747303,PrB_BB,Protein Bar,Noida 1,110001,337,In Stock,969385,158.333333
1,2023-03-30 14:59:29.206298,Muesli_P_CAC_350,Muesli,Noida 1,110001,296,In Stock,969385,158.000000
2,2023-03-30 14:59:33.364875,PrB_AF,Protein Bar,Noida 1,110001,115,In Stock,969385,166.250000
3,2023-03-30 14:59:45.936132,PrB_BB,Protein Bar,Delhi,110003,115,In Stock,1267916,110.833333
4,2023-03-30 15:00:05.411535,Muesli_P_CAC_350,Muesli,Delhi,110015,296,In Stock,1261842,118.500000
...,...,...,...,...,...,...,...,...,...
495,2023-03-30 19:06:20.402836,PB_DC400_Britannia_Bread,Peanut Butter,Kolkata,700086,0,Out of Stock,1167234,124.500000
496,2023-03-30 19:06:23.796389,Oats_DC_400,Oats,Kolkata,700086,180,In Stock,1167234,105.600000
497,2023-03-30 19:06:27.055669,Muesli_DC_400,Muesli,Kolkata,700086,309,In Stock,1167234,184.600000
498,2023-03-30 19:06:40.008910,EB_CC,Energy Bar,Kolkata,700089,108,Out of Stock,867467,116.533333
